In [1]:
%autosave 0

Autosave disabled


In [2]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from adam_prepare import titanic_pipeline

Let's read in our data using the titanic_pipeline function!

In [3]:
train, val, test = titanic_pipeline()

Let's take a peek at train. Some of my values are strings. Can (most) machine learning models handle strings?

In [4]:
train.head()

,survived,sex,age,sibsp,parch,fare,class,embark_town,alone
748,0,male,19.0,1,0,53.1000,First,Southampton,0
45,0,male,29.0,0,0,8.0500,Third,Southampton,1
28,1,female,29.0,0,0,7.8792,Third,Queenstown,1
633,0,male,29.0,0,0,0.0000,First,Southampton,1
403,0,male,28.0,1,0,15.8500,Third,Southampton,0


We need to determine a way to meaningfully encode our string values.

We will use [pd.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) to [one-hot encode](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/) our categorical columns.

Let's make sure our three subsets (train, validate, test) all have the same number of columns!

Now that our categorical data is properly encoded, let's make X and y dataframes for model training and evaluation.

We should define a baseline before we get started! Our baseline will be the most likely outcome from the y_train series.

We're ready to model! Let's create an instance of a [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) and fit it to our train dataset.

Let's check out the .feature_importances_ attribute.

Let's make some predictions with the .predict() method and save them to a variable.

Let's evaluate the accuracy of our model with the .score() method.

We can use the classification_report function to return a whole host of metrics about our model!

Wow! My model is incredible! I'm sure it will be equally awesome on the validate dataset!

What does it mean to be overfit?